<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/Insight-Sogang-Univ/insight-14th/blob/main/advanced/template/session05/assignment_languagemodel.ipynb" target="_parent">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
  </td>
</table>


# 🔍 이론 문제

### 1. SLM(통계적 언어모델)의 한계에 대해 설명하세요.

**[1번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답]
데이터가 부족해서 발생하는 희소 문제

### 2. BERT에 대한 설명이 아닌 것은?

a) 사전 학습 모델

b) 양방향 문맥 이해

c) 트랜스포머의 디코더를 쌓아 올린 구조

d) 이후 RoBERTa와 ALBERT로 발전

**[2번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답]
C, 인코더를 쌓아 올린 구조

### 3. BERT와 GPT의 학습 방식 차이를 간단히 설명하세요.

**[3번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답]
BERT는 양방향으로 이해 중심 학습을 진행하고, 트랜스포머의 인코더 부분으로 제작됨
GPT는 단뱡향으로 생성 중심 학습을 진행하고, 트랜스포머의 디코더 부분으로 제작됨

### 4. LLM이 지닌 한계와 RAG의 필요성에 대해 간단히 서술하세요.

**[4번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답] 세상에 새로운 단어가 지속적으로 창조되고 소멸되는 과정이 실시간으로 진행되기 때문에, 정보 검색의 과정을 추가하는 RAG가 필요

### 5. LLM을 경량화한 모델인 sLM을 만들기 위해 사용되는 대표적인 기술 3가지를 적으세요.(단답형)

**[5번 문제 정답]**

***여기에 정답을 써주세요!*** \
[정답] 프루닝, 지식증류, 양자화

# ✍ 언어 모델 과제

### 🧮 GPU 사용하기

In [1]:
import torch
import tensorflow as tf

# PyTorch GPU 상태
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ [PyTorch] Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")


# TensorFlow GPU 상태
print("\n✅ [TensorFlow] GPU Devices:")
print(tf.config.list_physical_devices('GPU'))

✅ [PyTorch] Using device: cuda
GPU name: Tesla T4

✅ [TensorFlow] GPU Devices:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 🔍 1. BERT

**BERT (Bidirectional Encoder Representations from Transformers)**
- 문장 분류(스팸 메일 탐지), 질의응답 시스템(챗봇, 검색 엔진), 번역(다국어 지원), 텍스트 요약(뉴스 요약) 등 다양한 NLP 작업에 사용
- 문맥을 양방향으로 이해하여 텍스트의 의미를 정밀하게 파악하며, 사전 학습된 모델을 기반으로 빠르게 응용 가능

### 1-1. 모델과 tokenizer 초기화
- tokenizer를 통해 문장을 토큰으로 나누고, 이를 정수 인덱스로 변환하여 모델이 이해할 수 있도록 함

In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
model = BertForMaskedLM.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
# tokenizer : raw text를 개별 토큰으로 분리(Wordpiece). 토크나이저 사전에 따라 고유한 정수 인덱스(고정값)를 매핑함.

# tokenizer를 통해 생성된 정수 인덱스 : 텍스트를 모델이 읽을 수 있도록 숫자화
# Embedding vector : 단어의 의미적, 문맥적 특성을 모델링

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 1-2. 입력 텍스트 준비 및 토큰화
- [MASK] 토큰을 삽입한 문장을 설정하고, `tokenizer.tokenize()`로 텍스트 토큰화
- [MASK]를 활용하여 문맥 속에서 특정 단어를 추론하는 상황을 만들고, 이를 모델이 학습한 패턴과 비교하도록 함


In [4]:
# 테스트할 문장
text = "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"

# 문장을 토큰으로 변환
tokenized_text = tokenizer.tokenize(text) # text(문장)을 토큰으로 변환해 tokenized_text에 저장합시다!
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!

print(tokenized_text) # 토큰이 출력됩니다.
print(indexed_tokens) # 인덱스가 출력됩니다.


['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '[MASK]', '을', '했', '##다', '.', '[SEP]']
[2, 1432, 2173, 3542, 16, 3616, 2259, 4084, 2079, 3934, 2069, 3790, 6895, 2062, 18, 3673, 10514, 3657, 16, 8983, 4, 1498, 1902, 2062, 18, 3]


### 1-3. MASK된 위치 확인
- [MASK] 토큰의 위치를 탐지해 해당 위치에서 모델이 단어를 예측하도록 지정합니다.


In [5]:
# 마스킹된 위치 찾기
masked_index = tokenized_text.index("[MASK]") # 토큰화된 결과물에서 index 메서드를 통해 [MASK]의 위치를 확인해봅시다!
print(masked_index)

20


### 1-4. 정수화된 토큰 인덱스를 tensor로 변환 후 모델 예측 실행
- 모델은 **텐서 데이터 구조**를 필요로 하므로, **입력값을 변환**하여 적합한 형식으로 만듦.
   - `torch.tensor(입력값 리스트 또는 입력값 ndarray)`를 통해 tensor 구조로 변환 가능!
- **역전파를 비활성화**하고, 모델이 **각 토큰에 대해 가능한 모든 단어 점수 출력**

In [6]:
# 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!
tokens_tensor = torch.tensor([indexed_tokens]) # 입력값(토큰 인덱스 리스트)을 tensor 구조로 변환해봅시다!

# 모델에 토큰 텐서를 전달하고 예측 실행
with torch.no_grad(): # 가중치 업데이트 없으므로 자동미분연산은 끔
    outputs = model(tokens_tensor) # 예측. 모델은 각 토큰위치에 대한 예측 결과를 반환.
    predictions = outputs[0] # logits (각 토큰위치에서 가능한 모든 토큰들의 원시점수)
print(predictions)

tensor([[[ -6.0337,   4.5526,  -5.6312,  ...,  -7.3959,  -7.4220,  -5.8337],
         [ -6.1762,   4.7585,  -7.3973,  ...,  -7.6209, -12.2124,  -6.0998],
         [ -7.4194,   3.9148,  -6.1517,  ...,  -6.8162,  -9.5421,  -3.0473],
         ...,
         [ -7.3761,   8.6972,  -5.4904,  ...,  -8.7202,  -9.1403,  -5.8566],
         [ -5.6347,  10.3884,  -4.3374,  ...,  -8.6900,  -7.4171,  -3.5043],
         [ -5.6652,  10.3407,  -4.4099,  ...,  -8.7521,  -7.4286,  -3.6681]]])


### 1-5. MASK된 토큰 예측
- [MASK] 위치에서 가장 높은 점수를 받은 토큰의 인덱스를 추출하고, 이를 단어로 변환

In [7]:
# 예측된 토큰 확인
predicted_index = torch.argmax(predictions[0, masked_index]).item() # masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

# 예측된 토큰으로 마스크 채우기
tokenized_text[masked_index] = predicted_token
# 토큰화된 텍스트를 다시 문자열로 변환
filled_text = tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

print("Original:", text)
print("Masked:", tokenized_text)
print("Predicted token:", predicted_token) ; print()

print("Filled sentence:", filled_text)

Original: [CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]
Masked: ['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '사랑', '을', '했', '##다', '.', '[SEP]']
Predicted token: 사랑

Filled sentence: 열여덟 , 우리는 서로의 이름을 처음 불렀다 . 그리고 스물 하나 , 우린 사랑 을 했다 .



### 1-6. MASK된 문장 복원
- 예측된 토큰으로 [MASK]를 대체하여 완성된 문장 생성

In [10]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertForMaskedLM.from_pretrained('klue/bert-base')

def fill_mask(input_text):
    # 텍스트를 토큰으로 변환
    tokenized_text = tokenizer.tokenize(input_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # 마스킹된 위치 찾기
    masked_index = tokenized_text.index("[MASK]") # 토큰화된 텍스트에서 마스킹된 위치를 찾아봅시다!

    # 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
    tokens_tensor = torch.tensor([indexed_tokens])

    # 모델에 토큰 텐서를 전달하고 예측 수행
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # 예측된 토큰 확인
    predicted_index = torch.argmax(predictions[0, masked_index]).item() # 위 코드 참고! masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    # 마스크를 채운 문장 반환
    tokenized_text[masked_index] = predicted_token # 토큰화된 텍스트의 마스킹된 위치(인덱스 사용)를 예측된 토큰으로 채워봅시다!
    return tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


예시 문장 만들어서 MLM을 사용해 보세요!

In [19]:
# 예시 1: "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"
# 예시 2: "[CLS] 윤해민은 INSGIHT 학회를 정말 사랑한다. 그는 INSIGHT 학회를 위해 [MASK]를 했다.[SEP]"
# 이 예시를 참고해서 [CLS](맨 앞), [MASK](채울 부분), [SEP](맨 뒤)를 추가해서 여러분만의 예시 문장 만들어주세요
input_text = "[CLS] 두산 베어스는 다음 야구 리그에서 모든 팀을 다 이긴 후 [MASK]을 차지할 것이다  [SEP]"
input_text_2 = "[CLS] 사랑을 했다, 우리가 만나. 지우지 못할 [MASK]이 되었다 [SEP]"
filled_text = fill_mask(input_text)
filled_text_2= fill_mask(input_text_2)
print("Filled sentence:", filled_text)
print("Filled sentence:", filled_text_2)

Filled sentence: 두산 베어스는 다음 야구 리그에서 모든 팀을 다 이긴 후 우승 을 차지할 것이다
Filled sentence: 사랑을 했다 , 우리가 만나 . 지우지 못할 추억 이 되었다


# 🔍 2. GPT, RAG, Langchain

## 2-1 라이브러리 설치

langchain 관련 라이브러리와, pdf 문서를 읽을수 있는 pypdf 라이브러리를 설치할게요!

In [17]:
# ⬇️ Colab에서 한 번만 실행
%pip install -qU "langchain>=0.3" langchain-community langchain-openai langgraph chromadb pypdf tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 26.3 MB/s eta 0:0

## 2-2. 키값 설정

## 2-3 pdf 문서 선택 (로드)


여러분들이 활용하고 싶은 아무 pdf 문서를 선택해보세요!

In [21]:
from google.colab import files
import os, re, unicodedata

os.makedirs("/content/pdfs", exist_ok=True)
print("📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).")
uploaded = files.upload()

name_map = {}
for i, (orig_name, data) in enumerate(uploaded.items(), 1):
    nf = unicodedata.normalize("NFC", orig_name)
    base, ext = os.path.splitext(nf)
    # 한글/영문/숫자/밑줄/대시/점만 남기고 나머지는 _
    safe_base = re.sub(r"[^\w\u3131-\u318E\uAC00-\uD7A3.\-]+", "_", base).strip("._'‘’“”")
    # 파일명 너무 길면 잘라냄 (최대 ~140자 정도로)
    safe_base = safe_base[:120] if len(safe_base) > 120 else safe_base
    new_name = f"doc_{i}_{safe_base}{ext.lower()}"
    with open(os.path.join("/content/pdfs", new_name), "wb") as f:
        f.write(data)
    name_map[new_name] = orig_name

print("✅ 저장 완료 (Colab 파일명 → 원본명 매핑):")
for k, v in name_map.items():
    print(f"  {k}  <=  {v}")


📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).


Saving AI 파도에 올라타는 법 (1).pdf to AI 파도에 올라타는 법 (1) (1).pdf
✅ 저장 완료 (Colab 파일명 → 원본명 매핑):
  doc_1_AI_파도에_올라타는_법_1_1.pdf  <=  AI 파도에 올라타는 법 (1) (1).pdf


## 2-4 문서 분할 후 벡터 DB에 저장
로딩한 문서를 분할 할 때,
- chunk_size는 `900`
- chunk_overlap은 `150`
- separators(구분자)는 `["\n\n", "\n", " ", ""]`
으로 설정하여 청킹해주세요!

In [23]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import glob

# 1) PDF → Documents
pdf_paths = sorted(glob.glob("/content/pdfs/*.pdf"))
docs = []
for p in pdf_paths:
    try:
        loader = PyPDFLoader(p)
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ 로딩 실패: {p} -> {e}")

print(f"총 문서 조각(페이지 기준): {len(docs)}")

# 2) 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 900, # 청크 사이즈: 문서를 몇 개의 토큰 단위로 나눌 것인지를 정합니다.
    chunk_overlap= 150, # 청크 오버랩: 분할된 끝 부분에서 맥락이 이어질 수 있도록 일부를 겹쳐서(overlap) 분할하는 것이 일반적입니다.
    separators= ["\n\n", "\n", " ", ""], # 구분자: 엔터, 공백 기준으로 나눔
)
chunks = text_splitter.split_documents(docs)
print(f"총 청크 수: {len(chunks)}")

# 3) 벡터스토어 (영속 디렉토리 쓰면 런타임 재시작해도 유지)
persist_dir = "/content/chroma_pdf_db"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 비용 절약형
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_dir
)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
print("✅ 인덱싱 완료")


총 문서 조각(페이지 기준): 275
총 청크 수: 275
✅ 인덱싱 완료


## 2-5 RA<U>Generate</U> : 생성 부분 선언

PROMPT 부분은 여러분들이 원하시는대로 입력해도 좋습니다!

🤥 영어로 입력하면 모델이 더 잘 이해한대요

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

PROMPT = ChatPromptTemplate.from_template(
"""다음 컨텍스트(업로드한 PDF에서 추출됨)만을 바탕으로 질문에 한국어로 답하라.
불확실하면 모른다고 말하라. 핵심 근거를 요약하고, 각 근거 옆에 출처(파일/페이지)를 표시하라.

[질문]
{question}

[컨텍스트]
{context}
이 pdf는 인공지능의 발전에 대한 기술들을 다루고 있다. 각 단원 별로 소개하고자 하는 내용에 집중해서, 각 내용을 이해해주기를 바란다.
특히 각 인공지능 기술 및 발전 과정에서 저자의 생각이 담겨 있는데, 이를 집중적으로 학습해야 한다.
"""
)

rag_chain = (PROMPT | llm | StrOutputParser())


## 2-6 RAG 파이프라인 langgraph로 구성

벡터db에서 검색하는 것이 <U>Retrieve</U>AG이고,

검색 결과를 생성 단계에 던져주는 것이 R<U>Augment</U>G입니다!

In [32]:
from typing import TypedDict, List
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

# format_docs의 역할은 문서를 포맷팅해서 모델에게 제공할 'context'를 정리하는 역할입니다!
def format_docs(docs):
    # 파일명/페이지를 메타데이터에 담아두는 PyPDFLoader 기본값 사용
    lines = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "")
        page = d.metadata.get("page", None)
        tag = f"{os.path.basename(src)}"
        if page is not None:
            tag += f" p.{page+1}"
        lines.append(f"[{i}] {tag}\n{d.page_content}")
    return "\n\n".join(lines)

# 그래프 선언
class QAState(TypedDict):
    question: str
    retrieved: List[Document]
    answer: str

def node_retrieve(state: QAState):
    q = state["question"]
    topk = retriever.invoke(q)
    return {"retrieved": topk}

def node_generate(state: QAState):
    q = state["question"]
    ctx = format_docs(state["retrieved"])
    ans = rag_chain.invoke({"question": q, "context": ctx})
    return {"answer": ans}

# --------- 그래프 구성 ---------
# 노드끼리 연결하기
workflow = StateGraph(QAState)
workflow.add_node("retrieve", node_retrieve) # 어떤 노드가 추가되어야 할까요?
workflow.add_node("generate", node_generate) # 어떤 노드가 추가되어야 할까요?

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate") # 어떤 노드와 어떤 노드가 연결되어야 할까요?
workflow.add_edge("generate", END)

app = workflow.compile()
print("✅ LangGraph 컴파일 완료")


✅ LangGraph 컴파일 완료


## 2-7 실제 구동해보기

question에 여러분들이 입력하고 싶은 질문을 넣어보세요!

RAG 과제는 이렇게 간단하게 끝내겠습니다😁

In [33]:
question = "이 책에서 다루는 내용을 간략하게 요약해줘 "
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
이 책은 인공지능(AI)의 발전과 그 원리를 다루고 있으며, 독자에게 AI 시대에서의 자신의 존재에 대해 고민하게 만드는 내용을 포함하고 있습니다. 주요 내용은 다음과 같습니다:

1. **AI의 구조와 원리**: 머신러닝, 딥러닝, 신경망, 생성형 모델 등 다양한 인공지능 기술의 구조와 원리를 설명하고, AI가 데이터를 분석하고 창작하는 방법을 탐구합니다. (출처: p.274)

2. **일반화의 개념**: AI가 새로운 데이터에서도 잘 판단할 수 있는 능력인 일반화에 대해 설명하며, 과소적합과 과적합의 개념을 통해 AI 학습의 복잡성을 다룹니다. (출처: p.11)

3. **딥러닝의 구조**: 딥러닝의 기본 구성 요소와 그 발전 과정을 설명하며, 기울기 소실 및 폭주 문제와 같은 딥러닝의 도전 과제를 다룹니다. (출처: p.53)

4. **데이터의 질서 탐구**: 데이터 안에 숨겨진 질서를 발견하기 위한 질문을 던지며, 오토인코더와 같은 기술을 통해 복잡한 데이터의 핵심을 추출하는 방법을 설명합니다. (출처: p.173)

5. **인간의 역할**: AI가 발전함에 따라 인간이 무엇을 할 수 있을지에 대한 질문을 제기하며, 기술이 창작의 일부를 대신하는 시대에서 인간의 창의성에 대한 고민을 강조합니다. (출처: p.274)

이 책은 AI 기술의 발전을 이해하고, 그에 따른 인간의 역할과 존재에 대한 성찰을 촉구하는 내용을 담고 있습니다.


In [34]:
question = "이 책에서 소개된 Transformer에 대해서 간략하게 설명해줘 "
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
트랜스포머(Transformer)는 기존의 순환 신경망(RNN)과는 다른 방식으로 데이터를 처리하는 신경망 구조입니다. 트랜스포머의 주요 특징은 데이터를 순서대로 하나씩 처리하는 것이 아니라, 모든 데이터를 동시에 한 번에 처리할 수 있다는 점입니다. 이로 인해 긴 문장에서도 정보의 손실 없이 효과적으로 처리할 수 있습니다. 또한, 트랜스포머는 중요한 정보에 집중하는 방식으로, 수많은 데이터 중에서 핵심적인 부분을 찾아내어 처리할 수 있는 능력을 가지고 있습니다. 이러한 특성 덕분에 최근의 챗봇 기술인 ChatGPT와 같은 다양한 인공지능 응용 프로그램에서 널리 사용되고 있습니다.

- 트랜스포머는 데이터를 동시에 처리하여 시간과 순서의 한계를 극복한다. (출처: doc_1_AI_파도에_올라타는_법_1_1.pdf, p.76)
- 트랜스포머는 중요한 정보에 집중하여 처리할 수 있는 신경망이다. (출처: doc_1_AI_파도에_올라타는_법_1_1.pdf, p.57)


In [35]:
question = "이 책을 바탕으로 생각해 보았을 때, 우리는 앞으로 어떤 태도로 인공지능을 마주해야 할까? "
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
앞으로 인공지능을 마주하는 태도는 주체적이고 질문 중심의 접근이 필요합니다. 다음은 그에 대한 핵심 근거입니다.

1. **질문을 통한 사고의 확장**: 인공지능이 발전함에 따라 우리는 "AI가 무엇을 할 수 있을까?"에서 "나는 무엇을 할 것인가?"라는 질문으로 나아가야 합니다. 이는 인간의 창의성과 상상력을 발휘하는 방향으로 나아가야 함을 의미합니다. (출처: doc_1_AI_파도에_올라타는_법_1_1.pdf p.274)

2. **기술의 이해와 활용**: 인공지능의 작동 원리를 이해하고, 이를 올바르게 활용하는 것이 중요합니다. 단순히 기술을 익히는 것을 넘어, 기술과 함께 살아가는 방법을 고민해야 합니다. (출처: doc_1_AI_파도에_올라타는_법_1_1.pdf p.4)

3. **인간의 상상력과 가능성의 확장**: 인공지능은 인간의 사고와 상상력을 확장하는 도구로 작용할 수 있습니다. 따라서 우리는 인공지능을 통해 새로운 가능성을 탐색하고, 더 정교하고 창의적인 시스템을 만들어 나가야 합니다. (출처: doc_1_AI_파도에_올라타는_법_1_1.pdf p.88)

4. **미래 지향적 사고**: 인공지능의 발전 방향을 예측하고, 기술이 어떻게 연결되고 사용될지를 고민하는 것이 중요합니다. 이는 단순한 기술 성능 향상에 그치지 않고, 우리가 함께 만들어갈 인공지능의 미래를 그리는 데 필요합니다. (출처: doc_1_AI_파도에_올라타는_법_1_1.pdf p.234)

이러한 태도를 통해 우리는 인공지능과의 관계를 더욱 의미 있게 발전시킬 수 있을 것입니다.


# 🔍 3. sLM

Copyright 2024 Google LLC.

## 3-1. sLM(Gemma) 로드
### sLM(small Language Model)의 대표 모델인 Gemma를 로드 해봅시다!

### (1) 사전 준비 사항
### 1. HuggingFace 로그인

https://huggingface.co/

이 링크로 들어가서 HuggingFace에 로그인합니다.(계정이 없다면 회원가입 해주세요!)

### 2. 토큰 발급받기

https://huggingface.co/settings/tokens

- 이 링크로 들어가서 'Create new token'을 클릭
- Token type: 'Read'로 해주세요.
- Token name: 아무거나 상관 없습니다.
- (⭐ 매우 중요!) 토큰 만드시고 **반드시 토큰 문자열 (hf_... 로 시작함)을 복사해주세요!**  
한 번만 볼 수 있습니다!

### 3. google colab에서 토큰 등록하기
- 왼쪽에 열쇠 모양 버튼(보안 비밀)을 눌러주세요.
- 이름: colab-gemma
- 값: 복사해둔 토큰 문자열(hf_...로 시작함)
- 노트북 액세스 : 스위치 키기(파란색)

### 4. gemma-2b-it 모델 사용 약관 동의

https://huggingface.co/google/gemma-2b-it

- (⭐ 매우 중요!) 반드시 2번 단계에서 토큰을 발급받았던 바로 그 Hugging Face 계정으로 로그인된 상태에서 위 링크로 들어가야 합니다.
- 링크로 들어가서 'Access Gemma on Hugging Face' 부분의 약관을 읽고 동의 버튼을 클릭해 주세요.
- "You have been granted access to this model"이라는 메시지가 뜨면 성공입니다.

### (2) 필요한 툴 설치

In [3]:
!pip install -U bitsandbytes
!pip install --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.8 MB/s eta 0:00:00


In [2]:
# HuggingFace에 로그인합니다.

from huggingface_hub import login
from google.colab import userdata

# 따음표 안에 본인의 보안 비밀 키의 이름을 적습니다.(여기선 colab-gemma)
try:
    login(token=userdata.get("colab-gemma"))
except Exception as e:
    print(f"Error logging in to Hugging Face: {e}")
    print("Please make sure you have added your Hugging Face token to Colab secrets with the name 'colab-gemma'")

In [3]:
# 모델을 로드합니다.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 계산 부하를 줄이고 추론 속도를 높이기 위해 '양자화'합니다.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# sLM(gemma)를 Q&A, 요약 등에 활용하기 위해 '지시 튜닝(Instruction-Tuned) 버전의 모델을 사용합니다.
# "google/gemma-2b-it" 모델을 사용합니다.
model_id = "google/gemma-2b-it"

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델을 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # 양자화(quantization)
    device_map={"":"cuda:0"} # Changed from "0" to "cuda:0"
)

# GPU를 사용하도록 설정합니다.
device = "cuda"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## 3-2. sLM 활용
- sLM은 LLM을 경량화한 모델이기 때문에 언어 모델의 기능인 질의응답(추론), 요약, 분류(감정분석) 등이 가능합니다.
- **속도는 비교적 빠르지만, 성능이 그닥 좋지 않다는 것을 확인하실 수 있습니다.**

### Gemma 활용 예시 (Common Use Cases)


### (1) 질의응답(Reasoning)

#### 원하는 질문을 한 번 넣어보세요!

In [4]:
print("--- 예시 1: 질의응답 (Reasoning) ---")

# content : '~~' 따음표 안에 질문을 써주시면 됩니다.
# 하고 싶은 질문 아무거나 넣어보세요!

# "role": "user" 입니다.

chat = [
    # Gemma 모델의 채팅 형식에 맞춰 'role'과 'content'를 지정합니다.
    { "role": "user", "content": "AI가 회계 분야에서 어떻게 활용될 수 있는지 알려줘" }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성)
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)

--- 예시 1: 질의응답 (Reasoning) ---
**AI가 회계 분야에서 어떻게 활용될 수 있는지**

**1. 데이터 분석 및 예측**
* AI는 회계 데이터를 분석하고 예측하여 미래 결과를 추측할 수 있습니다.
* 예를 들어, 회계 수익과 비용을 예측하고, 회계 성과를 개선하기 위한 전략을 수립할 수 있습니다.

**2. 회계 자동화**
* AI는 회계 업무를 자동화할 수 있는 도구를 개발할 수 있습니다.
* 예를 들어, 데이터 입력, 계정 관리, 보고서 생성 등을 자동화할 수 있습니다.

**3. 회계 의사 결정 지원**
* AI는 회계 의사 결정에 도움을 줄 수 있는 정보와 분석을 제공할 수 있습니다.
* 예를 들어, 회계 성과와 리스크를 평가하고, 투자 및 재무 계획을 수립할 수 있습니다.

**4. 회계 솔루션 제공**
* AI는 회계 솔루션을 제공하는 데 사용될 수 있습니다.
* 예를 들어, 회계 시스템 구축, 데이터


### (2) 요약 (Summarization)

#### 원하는 글을 넣어서 요약해 보세요!

In [5]:
print("\n--- 예시 2: 요약 (Summarization) ---")

# text_to_summarize = """~~~""" 따음표 안에 요약하고 싶은 글을 넣어주시면 됩니다.""
# 요약하고 싶은 글 아무거나 넣어보세요!

text_to_summarize = """
1. 가스 스트리핑 (Gas Stripping)
C1 가스 발효에서 가장 적용하기 쉽고 자연스러운 기술입니다.

메커니즘:

발효조 하부로 가스(N₂, CO₂ 또는 미반응 합성가스)를 주입하면 가스 기포가 배양액을 통과하며 상승합니다.

이때 휘발성이 강한 생산물(에탄올, 부탄올, 아세톤 등)이 기-액 평형(Vapor-Liquid Equilibrium) 원리에 의해 액체에서 가스 기포 쪽으로 물질 전달(Mass Transfer)이 일어납니다.

생산물을 머금은 가스는 발효조 상부로 배출되고, **응축기(Condenser)**를 통과하며 냉각되어 액체 상태의 고농도 생산물로 회수됩니다.

공정 구성의 특징:

가스 재순환: 회수 후 남은 가스는 다시 발효조로 순환(Recycle)시켜 스트리핑 가스로 재사용하거나 미생물의 먹이로 씁니다.

장점: 별도의 외부 장치가 크게 필요 없고, 미생물에 대한 물리적/화학적 스트레스가 거의 없습니다.

단점: 물과 생산물의 분리 효율(Selectivity)이 낮아, 물도 함께 증발하므로 응축액 내 알코올 농도가 아주 높지는 않습니다. 후단에 추가 증류가 필요합니다.

2. 투과증발 (Pervaporation)
에너지 효율이 가장 우수하여 최근 고부가가치 연구에서 주목받는 기술입니다.

메커니즘:

선택적 투과: 배양액을 특정 성분(알코올)만 잘 통과시키는 **분리막(Membrane)**과 접촉시킵니다.

추진력(Driving Force): 막의 반대편(투과부)을 진공 상태로 만들거나 **비활성 가스(Sweep gas)**를 흘려보내 증기압 차이를 만듭니다.

이 압력 차이로 인해 알코올 분자가 액체 상태로 막 표면에 흡착 -> 막 내부 확산 -> 기체 상태로 증발하여 반대편으로 빠져나갑니다.

공정 구성의 특징:

소수성 분리막(Hydrophobic Membrane): 물은 배척하고 유기 용매만 통과시키는 실리콘 고무(PDMS) 등의 재질을 사용합니다.

장점: 분리 효율(Selectivity)이 매우 높아 99% 이상의 고순도 용매를 얻을 수 있으며, 증류로는 분리가 힘든 **공비점(Azeotrope)**을 깰 수 있습니다.

단점: 배양액 속의 세포나 단백질이 막 표면을 막는 파울링(Fouling) 현상이 발생하여 주기적인 세척이나 교체가 필요합니다.

3. 액-액 추출 (Liquid-Liquid Extraction, LLE)
물과 섞이지 않는 유기 용매를 이용하는 전통적인 화학공학적 방법입니다.

메커니즘:

배양액에 생산물을 잘 녹이는 **추출제(Extractant, 유기 용매)**를 접촉시킵니다.

분배 계수(Partition Coefficient): 생산물이 물보다 추출제 쪽에 더 잘 녹아들어가는 성질을 이용해, 생산물이 추출제 층으로 이동합니다.

이후 층 분리된 추출제를 꺼내어 증류 등을 통해 생산물을 회수하고, 추출제는 다시 사용합니다.

공정 구성의 특징:

생체 적합성(Biocompatibility): 추출제 자체가 미생물에게 독성을 띠면 안 됩니다. (예: Oleyl alcohol 등 독성이 낮은 용매 사용)

장점: 분리 용량이 크고 공정이 비교적 단순합니다.

단점: 용매가 물에 아주 조금이라도 녹아 발효조에 남으면 미생물 활성을 떨어뜨릴 수 있고, 용매와 배양액이 섞여 에멀전(Emulsion)을 형성하면 층 분리가 어려워집니다.

4. 흡착 (Adsorption)
고체 입자에 생산물을 달라붙게 하는 방식입니다.

메커니즘:

배양액을 **흡착제(Adsorbent)**가 채워진 컬럼(Column)에 통과시킵니다.

특정 분자(알코올)만 선택적으로 흡착제의 미세 기공에 포집됩니다.

흡착제가 포화되면 열이나 압력을 가해 생산물을 떼어내는 탈착(Desorption) 과정을 거쳐 회수합니다.

공정 구성의 특징:

재료: 활성탄(Activated Carbon), 제올라이트(Zeolite), 고분자 수지(Resin) 등이 사용됩니다. 소수성 제올라이트(Silicalite)가 알코올 회수에 효과적입니다.

장점: 에너지 소모가 적고, 미생물 독성 문제가 없습니다.

단점: 세포가 흡착제 기공을 막을 수 있어, 보통 세포를 먼저 걸러낸 후(여과) 상등액만 흡착 컬럼으로 보냅니다. 탈착 과정에서 에너지가 추가로 듭니다.
"""

# content : '~~~'에 요청사항을 적으시면 됩니다. 여기서는 위 글을 세 문장으로 요약해 달라고 요청했습니다.
# "role": "user" 입니다.

chat = [
    { "role": "user", "content": f"Could you summarize the following text in four paragraph?\n\nText:\n{text_to_summarize}" }
]

# 여기부터는 각 사례 모두 코드가 동일합니다.

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 2: 요약 (Summarization) ---
1. 가스 스트리핑은 가장 적용하기 쉽고 자연스러운 기술로, 발효조 하부로 가스(N₂, CO₂, 또는 미반응 합성가스)를 주입하면 가스 기포가 배양액을 통과하며 상승합니다.

2. 투과증발은 에너지 효율이 가장 우수하여 최근 고부가가치 연구에서 주목받는 기술로, 선택적 투과를 통해 99% 이상의 고순도 용매를 얻을 수 있습니다.

3. 액-액 추출은 물과 섞이지 않는 유기 용매를 이용하는 전통적인 화학공학적 방법으로, 생산물을 회수하고, 추출제는 다시 사용합니다.

4. 흡착은 고체 입자에 생산물을 달라붙게 하는 방식으로, 재료를 선택적으로 흡착하여 회수합니다.


### (3) 분류 (Classification)

원하는 문장을 넣고 감정 분석을 해보세요!

In [14]:
print("\n--- 예시 3: 분류 (Classification) ---")

# content: '~~~' 따음표 안에 요청사항을 적으시면 됩니다.
# 여기서는 긍정, 부정, 중립으로 감정분석을 요청했습니다.

# \n\nText: 여기에 감정분석할 문장을 적어주시면 됩니다.
# 원하는 문장을 넣고 감정 분석을 해보세요!

# "role": "user" 입니다.

chat = [
    { "role": "user", "content": "Classify the text into neutral, negative, or positive. Generate only the class, nothing else.\n\nText: 진짜 너무너무 행복해 즐거워 아름다워 모든 것이 좋아  " }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 3: 분류 (Classification) ---
Positive.

The text expresses a positive sentiment, indicating happiness and enjoyment.



#### sLM 활용사례 결과 요약

- sLM은 질의응답이나 요약은 어느 정도 수행하지만, 퀄리티가 아쉽습니다.
- 감정 분석에서 '너무 재미가 없다'를 중립으로 판단하는 것으로 보아 분류 작업에서도 한계가 나타납니다.


- 이렇게 범용적인 활용에서는 한계가 나타나지만, 미세조정(Fine-Tuning)을 통해 특정 작업에 특화하면 해당 작업에는 좋은 성능을 내면서도 여전히 경량화된 모델의 장점을 누릴 수 있는 것이 sLM의 특징입니다!

# 🤩🤩 수고하셨습니다 🤩🤩
⭐️⭐️ 2-2의 API KEY 셀을 지우고 과제 제출해주세요!! ⭐️⭐️